In [14]:
import pandas as pd
import mysql.connector

# List of CSV file paths["bus_det_final_csvfile_2_kaac.csv"]
csv_files = ["bus_det_final_csvfile_1_hrtc.csv",
             "bus_det_final_csvfile_2_kaac.csv",
             "bus_det_final_csvfile_3_meghalaya.csv",
             "bus_det_final_csvfile_4_assam_astc.csv",
             "bus_det_final_csvfile_5_kadamba_ktcl.csv",
             "bus_det_final_csvfile_6_andhra_apsrtc.csv",
             "bus_det_final_csvfile_7_bihar_bsrtc.csv",
             "bus_det_final_csvfile_8_ksrtc_kerala.csv",
             "bus_det_final_csvfile_9_punjab_pepsu.csv",
             "bus_det_final_csvfile_10_telaungana.csv"
            ]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)
combined_df = combined_df.dropna(subset=['Bus_Name'])


# Convert departing time and reaching time into time format, replacing NaT with None
# Convert departing time and reaching time into time format, replacing invalid times with None
#combined_df['Departing_Time'] = pd.to_datetime(combined_df['Departing_Time'], format='%H:%M:%S', errors='coerce').dt.time
#combined_df['Reaching_Time'] = pd.to_datetime(combined_df['Reaching_Time'], format='%H:%M:%S', errors='coerce').dt.time


# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Red_bus_project"
)

mycursor = mydb.cursor()






# Create table query
create_table_query = """CREATE TABLE bus_routes1(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time TIME NOT NULL,
    duration TEXT NOT NULL,
    reaching_time TIME NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """

mycursor.execute(create_table_query)

# Insert query (ensure that the column names match exactly with the DataFrame)
insert_query = """
INSERT INTO bus_routes1 (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame, ensuring None is passed for missing times
for i, row in combined_df.iterrows():
    mycursor.execute(insert_query, tuple(row))

# Commit the changes
mydb.commit()

print("Data inserted successfully.")


Data inserted successfully.


In [13]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Red_bus_project"
)

mycursor = mydb.cursor()


In [19]:
mycursor.execute("SELECT count(*) FROM bus_routes2")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


+------------+
|   count(*) |
|------------|
|       3687 |
+------------+


In [21]:
import pandas as pd
import mysql.connector
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# List of CSV file paths
csv_files = [
    "bus_det_final_csvfile_1_hrtc.csv",
    "bus_det_final_csvfile_2_kaac.csv",
    "bus_det_final_csvfile_3_meghalaya.csv",
    "bus_det_final_csvfile_4_assam_astc.csv",
    "bus_det_final_csvfile_5_kadamba_ktcl.csv",
    "bus_det_final_csvfile_6_andhra_apsrtc.csv",
    "bus_det_final_csvfile_7_bihar_bsrtc.csv",
    "bus_det_final_csvfile_8_ksrtc_kerala.csv",
    "bus_det_final_csvfile_9_punjab_pepsu.csv",
    "bus_det_final_csvfile_10_telaungana.csv"
]

# Read each CSV file into a DataFrame and concatenate them
try:
    df_list = [pd.read_csv(file) for file in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    logging.info("CSV files read and combined successfully.")
except Exception as e:
    logging.error(f"Error reading CSV files: {e}")
    raise

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].fillna(0).astype(int)
combined_df = combined_df.dropna(subset=['Bus_Name'])

# Convert time columns if available
combined_df['Departing_Time'] = pd.to_datetime(combined_df['Departing_Time'], format='%H:%M:%S', errors='coerce').dt.time
combined_df['Reaching_Time'] = pd.to_datetime(combined_df['Reaching_Time'], format='%H:%M:%S', errors='coerce').dt.time

# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="Red_bus_project"
    )
    mycursor = mydb.cursor()
    logging.info("Connected to MySQL database.")
except mysql.connector.Error as err:
    logging.error(f"Error connecting to MySQL: {err}")
    raise

# Create table query with IF NOT EXISTS
create_table_query = """CREATE TABLE IF NOT EXISTS redbus_routes(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time TIME,
    duration TEXT NOT NULL,
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
"""

try:
    mycursor.execute(create_table_query)
    logging.info("Table created successfully (if it did not already exist).")
except mysql.connector.Error as err:
    logging.error(f"Error creating table: {err}")
    mycursor.close()
    mydb.close()
    raise

# Insert query
insert_query = """
INSERT INTO redbus_routes(id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame
for i, row in combined_df.iterrows():
    values = tuple(row.where(pd.notnull(row), None))  # Replace NaN with None
    try:
        mycursor.execute(insert_query, values)
    except mysql.connector.Error as err:
        logging.error(f"Error inserting row {i + 1}: {err}")

# Commit the changes
try:
    mydb.commit()
    logging.info("Data committed successfully.")
except mysql.connector.Error as err:
    logging.error(f"Error committing data: {err}")

# Close the cursor and connection
mycursor.close()
mydb.close()
logging.info("Database connection closed.")

print("Data inserted successfully.")


2024-10-08 15:51:30,066 - INFO - CSV files read and combined successfully.
2024-10-08 15:51:30,083 - INFO - Connected to MySQL database.
2024-10-08 15:51:30,110 - INFO - Table created successfully (if it did not already exist).
2024-10-08 15:51:30,846 - INFO - Data committed successfully.
2024-10-08 15:51:30,846 - INFO - Database connection closed.


Data inserted successfully.
